# Multiscale MILP

This is a continuation of 'One Location, One Temporal Scale, One Operation, Linear Programming Example' [Example 1]. Refer Example 1 to learn the basics on how Energia models processes. 

In this example, we add another Process [Solar PV] and Storage [Li-ion Battery]. Technology choice is modeled using binaries. Moreover, the model is multiscale as the operational capacities are decision variables. 

In [19]:
from energia import *

m = Model('example2')
m.q = Periods()
m.y = 4 * m.q
m.usd = Currency()

## Resources

### Conveniently declaring components

Use m.declare(\<Object Type\>, \<list of names\>) to declare a large number of objects in one step.

In [20]:
m.declare(Resource, ['power', 'wind', 'solar'])

### Set bounds on Resource flows

Unlike wind which has bound on the total consumption, we set a daily limit on solar energy. The same bound is repeated in each quarter. The following constraints are written.

$\mathbf{cons}_{solar, network, quarter_0} \leq 100$

$\mathbf{cons}_{solar, network, quarter_1} \leq 100$

$\mathbf{cons}_{solar, network, quarter_2} \leq 100$

$\mathbf{cons}_{solar, network, quarter_3} \leq 100$

In [21]:
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

2025-10-21 13:07:39,808 [INFO] Balance for solar in (l, q): initializing
2025-10-21 13:07:39,809 [INFO] ✔ Completed in 0.0001628398895263672 seconds
2025-10-21 13:07:39,809 [INFO] Binding consume in domain (solar, l, q)
2025-10-21 13:07:39,810 [INFO] ✔ Completed in 0.00010728836059570312 seconds
2025-10-21 13:07:39,810 [INFO] Balance for wind in (l, y): initializing
2025-10-21 13:07:39,810 [INFO] ✔ Completed in 0.00010156631469726562 seconds
2025-10-21 13:07:39,811 [INFO] Binding consume in domain (wind, l, y)
2025-10-21 13:07:39,811 [INFO] ✔ Completed in 6.318092346191406e-05 seconds
2025-10-21 13:07:39,811 [INFO] Balance for power in (l, q): initializing
2025-10-21 13:07:39,813 [INFO] ✔ Completed in 0.0001392364501953125 seconds
2025-10-21 13:07:39,813 [INFO] Binding release in domain (power, l, q)
2025-10-21 13:07:39,814 [INFO] ✔ Completed in 0.00015234947204589844 seconds


## Operations 

### Capacity as a variable 

Here we want the optimization problem to determine the optimal capacity. Moreover, we set binaries to avoid the lower bound being adhered to if the process is not set up. 

If the bounds are meant to be compulsory limits, skip the .x 

In [22]:
m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.capacity.show()

2025-10-21 13:07:39,818 [INFO] Binding capacity in domain (wf, l, y)
2025-10-21 13:07:39,819 [INFO] ✔ Completed in 0.0001804828643798828 seconds
2025-10-21 13:07:39,819 [INFO] Binding capacity in domain (wf, l, y)
2025-10-21 13:07:39,820 [INFO] ✔ Completed in 0.0004088878631591797 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

Unlike in Example 1, where the capacity was know, capacity is a variable here. 

Moreover, the expenditure associated with operating and capacitating are different

In [23]:
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49
m.operate.show(True)

2025-10-21 13:07:39,826 [INFO] Binding operate in domain (wf, l, q)
2025-10-21 13:07:39,827 [INFO] ✔ Completed in 0.00021195411682128906 seconds
2025-10-21 13:07:39,828 [INFO] Mapping operate: (wf, l, q) → (wf, l, y)
2025-10-21 13:07:39,828 [INFO] ✔ Completed in 0.00010967254638671875 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [24]:
m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10

m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

2025-10-21 13:07:39,837 [INFO] Binding capacity in domain (pv, l, y)
2025-10-21 13:07:39,838 [INFO] ✔ Completed in 0.0001888275146484375 seconds
2025-10-21 13:07:39,838 [INFO] Binding capacity in domain (pv, l, y)
2025-10-21 13:07:39,839 [INFO] ✔ Completed in 0.00014209747314453125 seconds
2025-10-21 13:07:39,839 [INFO] Binding operate in domain (pv, l, q)
2025-10-21 13:07:39,840 [INFO] ✔ Completed in 0.00019788742065429688 seconds
2025-10-21 13:07:39,841 [INFO] Mapping operate: (pv, l, q) → (pv, l, y)
2025-10-21 13:07:39,841 [INFO] ✔ Completed in 0.00010013580322265625 seconds


### Storage Operation

energia now allows storing to require the use of other resources, example power for hydrogen cryogenic storage. 

Provide an equation similar to Process, in this case the basis is the stored resource 
If no other resource is provided, it is assumed to be the charging/discharging efficiency

Note that the following are created internally: 
1. auxilary resource  with name resource.stored 
2. charging and discharging processes as storage.charge and storage.discharge 

The parameters for each of these can be set individually, thus allowing for a wide range of modeling approaches 

In [25]:
m.lii = Storage()

m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10

# m.lii.capacity >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432

m.lii.inventory[m.usd.spend] == 2000
m.lii.charge.capacity <= 100
m.lii.discharge.capacity <= 100

2025-10-21 13:07:39,846 [INFO] Binding invcapacity in domain (power.lii, l, y)
2025-10-21 13:07:39,846 [INFO] ✔ Completed in 0.0001747608184814453 seconds
2025-10-21 13:07:39,847 [INFO] Binding invcapacity in domain (power.lii, l, y)
2025-10-21 13:07:39,848 [INFO] ✔ Completed in 0.00015115737915039062 seconds
2025-10-21 13:07:39,848 [INFO] Balance for power.lii in (l, y): initializing
2025-10-21 13:07:39,849 [INFO] Binding capacity in domain (lii.charge, l, y)
2025-10-21 13:07:39,849 [INFO] ✔ Completed in 5.9604644775390625e-05 seconds
2025-10-21 13:07:39,850 [INFO] Binding capacity in domain (lii.discharge, l, y)
2025-10-21 13:07:39,850 [INFO] ✔ Completed in 5.984306335449219e-05 seconds


## Locating Operations

Operations can be located as 

operation.locate(\<list of locations\>)

or 

m.location.operations(\<list of operations\>)

They both do the same thing 

In [26]:
m.pem = Process()

m.pem(-m.power) == m.solar 

In [27]:
m.pv.locate(m.network)
m.network.locate(m.wf, m.lii)

2025-10-21 13:07:39,859 [INFO] Balance for power in (l, q): adding produce(power, l, q, operate, pv)
2025-10-21 13:07:39,860 [INFO] ✔ Completed in 0.00014853477478027344 seconds
2025-10-21 13:07:39,860 [INFO] Balance for solar in (l, q): adding expend(solar, l, q, operate, pv)
2025-10-21 13:07:39,861 [INFO] ✔ Completed in 0.00013256072998046875 seconds
2025-10-21 13:07:39,862 [INFO] Balance for power in (l, q): adding produce(power, l, q, operate, wf)
2025-10-21 13:07:39,862 [INFO] ✔ Completed in 0.00015115737915039062 seconds
2025-10-21 13:07:39,863 [INFO] Balance for wind in (l, y): adding expend(wind, l, y, operate, wf)
2025-10-21 13:07:39,863 [INFO] ✔ Completed in 9.989738464355469e-05 seconds
2025-10-21 13:07:39,864 [INFO] Assuming  power.lii inventory capacity is unbounded in (l, y)
2025-10-21 13:07:39,864 [INFO] Assuming inventory of power.lii is unbounded in (l, y)
2025-10-21 13:07:39,864 [INFO] Assuming inventory of power.lii is bound by inventory capacity in (l, q)
2025-10-21

## Inventory Balance

Inventory is passed on from one time period (t - 1) to the next (t) and hence features in the general resource balance for resource.stored 

In [28]:
m.inventory.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Optimize!

In [29]:
m.usd.spend.opt()

2025-10-21 13:07:39,885 [INFO] Mapping spend: (usd, l, y, capacity, wf) → (usd, l, y)
2025-10-21 13:07:39,885 [INFO] ✔ Completed in 0.00012922286987304688 seconds
2025-10-21 13:07:39,886 [INFO] Mapping spend: (usd, l, y, operate, wf) → (usd, l, y)
2025-10-21 13:07:39,886 [INFO] ✔ Completed in 0.00011372566223144531 seconds
2025-10-21 13:07:39,886 [INFO] Mapping spend: (usd, l, y, capacity, pv) → (usd, l, y)
2025-10-21 13:07:39,887 [INFO] ✔ Completed in 0.00011157989501953125 seconds
2025-10-21 13:07:39,888 [INFO] Mapping spend: (usd, l, y, operate, pv) → (usd, l, y)
2025-10-21 13:07:39,888 [INFO] ✔ Completed in 0.00011420249938964844 seconds
2025-10-21 13:07:39,888 [INFO] Mapping spend: (usd, l, y, invcapacity, power.lii) → (usd, l, y)
2025-10-21 13:07:39,889 [INFO] ✔ Completed in 8.606910705566406e-05 seconds
2025-10-21 13:07:39,889 [INFO] Mapping spend: (usd, l, y, inventory, power.lii) → (usd, l, y)
2025-10-21 13:07:39,889 [INFO] ✔ Completed in 9.489059448242188e-05 seconds
2025-10-

Read MPS format model from file Program(example2).mps
Reading time = 0.00 seconds
PROGRAM(EXAMPLE2): 85 rows, 78 columns, 198 nonzeros


2025-10-21 13:07:39,895 [INFO] Optimizing Program(example2) using gurobi


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 85 rows, 78 columns and 198 nonzeros
Model fingerprint: 0x2d658939
Variable types: 75 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 4e+02]
Presolve removed 70 rows and 63 columns
Presolve time: 0.00s
Presolved: 15 rows, 15 columns, 44 nonzeros
Variable types: 15 continuous, 0 integer (0 binary)

Root relaxation: objective 3.006497e+08, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3.006497e+08 3.0065e+08  0.0

2025-10-21 13:07:39,905 [INFO] Solution found. Use .sol() to display it
2025-10-21 13:07:39,906 [INFO] Creating Solution object, check.solution


## Solution

### Inventory Profiles

The inventory maintained in each time period is:

In [30]:
m.inventory.sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount charged into inventory is:

In [31]:
m.produce(m.power.lii, m.lii.charge.operate, m.q).sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount discharged from inventory is:

In [32]:
m.produce(m.power, m.lii.discharge.operate, m.q).sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Integer Decisions 

All the operations are setup in this case

In [33]:
m.capacity.reporting.sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [34]:
m.capacity.sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [35]:
m.show(True)

# Mathematical Program for Program(example2)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###  Functions

<IPython.core.display.Math object>